<a href="https://colab.research.google.com/github/frank061635/test/blob/master/MLP_keras_HT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Model 구축 관련 import
import pandas as pd
import keras
from sklearn.preprocessing import minmax_scale
import numpy as np   
from numpy import argmax
from keras.models import Sequential,load_model
from keras.layers import Dense, Activation
from keras import optimizers
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split

In [5]:
!pip install xlrd
!pip install psycopg2

# Train, Test 데이터 Load
#raw data format: 
#ali(A to G///fluid,diameter(mm),temperature(C),mass velocity(kg/m^2s),quality,h(W/m^2s), heatflux(W/cm^2) )
#agostini(A to H///fluid,diameter(mm),temperature(C),mass velocity(kg/m^2s),quality,h(W/m^2s), heatflux(W/cm^2),aspect ratio )

#paper
# split into train- ing and test datasets with 75% – 25% split ratio 

excel_dir_1 = "https://github.com/frank061635/test/raw/master/Consolidate_modified.xlsx"
#blob-->raw로 변경해야 파일 다운로드 가능/ header=1
sample_Co2= pd.read_excel(excel_dir_1, sheet_name='Co2', header=1)
sample_FC72= pd.read_excel(excel_dir_1, sheet_name='FC-72', header=1)
sample_Water= pd.read_excel(excel_dir_1, sheet_name='Water', header=1)
sample_R1234ze= pd.read_excel(excel_dir_1, sheet_name='R1234ze', header=1)
sample_R1234yf= pd.read_excel(excel_dir_1, sheet_name='R1234yf', header=1)
sample_R245fa= pd.read_excel(excel_dir_1, sheet_name='R245fa', header=1)
sample_R236fa= pd.read_excel(excel_dir_1, sheet_name='R236fa', header=1)
sample_R152= pd.read_excel(excel_dir_1, sheet_name='R152', header=1)
sample_R134a= pd.read_excel(excel_dir_1, sheet_name='R134a', header=1)
sample_R123= pd.read_excel(excel_dir_1, sheet_name='R123', header=1)
sample_R113= pd.read_excel(excel_dir_1, sheet_name='R113', header=1)
sample_R22= pd.read_excel(excel_dir_1, sheet_name='R22', header=1)
sample_R32= pd.read_excel(excel_dir_1, sheet_name='R32', header=1)

num_Co2= sample_Co2.shape[0] 
num_FC72= sample_FC72.shape[0] 
num_Water= sample_Water.shape[0] 
num_R1234ze= sample_R1234ze.shape[0] 
num_R1234yf= sample_R1234yf.shape[0] 
num_R245fa= sample_R245fa.shape[0] 
num_R236fa= sample_R236fa.shape[0] 
num_R152= sample_R152.shape[0] 
num_R134a= sample_R134a.shape[0] 
num_R123= sample_R123.shape[0] 
num_R113= sample_R113.shape[0] 
num_R22= sample_R22.shape[0] 
num_R32= sample_R32.shape[0] 

print("num of train samples_Co2: ", num_Co2)

sample_R134a
#sample_1["fluid"][130] ==> Fluid column의 130번째 행의 값, 실제엑셀 2행-->0행
#sample_2= pd.read_excel(excel_dir, sheet_name='Sheet1')
#num_of_train_samples_2 = sample_2.shape[0] 
#print("num of train samples_2: ", num_of_train_samples_2)
#sample_2

num of train samples_Co2:  2449


NameError: ignored

In [ ]:
# 데이터 병합
#paper
# standardized to a Gaussian like distribution with zero mean value and unit variance
# sample의 데이터를 모두 병합한 consolidate_sample
#x_train==> (인풋데이터 갯수,19) y_train==> (인풋데이터 갯수,1)==> h_exp(참값)
consolidate_sample=pd.concat([sample_Co2,sample_FC72,sample_Water,sample_R1234ze,sample_R1234yf,
                              sample_R245fa,sample_R236fa,sample_R152,sample_R134a,sample_R123,
                              sample_R113,sample_R22,sample_R32]) #두 데이터 연결한 데이터 셋 생성 consolidate_sample=pd.concat([sample_1,sample_2,3,4-----])
consolidate_sample=consolidate_sample.astype(np.float64)
X_train=consolidate_sample
X_train=X_train.drop(['h_exp'],axis=1)
y_train=consolidate_sample['h_exp']
# 데이터 스플릿 test_size= 테스트 비율 
# X_train, X_test , y_train, y_test = train_test_split(X_train, y_train, test_size = 0.25, random_state = 7)
_,X_test,_,y_test = train_test_split(X_train, y_train, test_size = 0.30, random_state = 7)


In [ ]:
#데이터 특성분석
train_stats = X_train.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Co,152.0,4.540985e-02,8.924845e-02,6.213524e-04,6.538076e-03,1.504395e-02,3.241646e-02,6.759962e-01
Bd,152.0,2.047151e+00,6.663712e-04,2.046644e+00,2.046644e+00,2.046644e+00,2.048021e+00,2.048021e+00
Bo,152.0,1.024986e-03,5.170397e-04,1.433041e-04,5.729540e-04,1.145908e-03,1.432385e-03,2.005339e-03
Frf,152.0,1.768405e+00,2.414491e+00,1.887778e-03,2.016771e-01,6.477646e-01,2.207296e+00,8.047148e+00
Frfo,152.0,2.784014e+00,3.200503e+00,1.268327e-01,9.019213e-01,9.023895e-01,5.637740e+00,8.117292e+00
Frg,152.0,5.876567e+08,1.373738e+09,7.577345e+03,1.207089e+07,7.674806e+07,3.459010e+08,8.608325e+09
Frgo,152.0,6.187168e+09,7.153476e+09,2.833350e+08,1.952471e+09,1.952471e+09,1.259267e+10,1.813344e+10
Prf,152.0,2.866321e+01,5.240380e-02,2.859483e+01,2.859483e+01,2.870311e+01,2.870311e+01,2.870311e+01
Prg,152.0,1.013838e+00,3.536964e-04,1.013376e+00,1.013376e+00,1.014107e+00,1.014107e+00,1.014107e+00
Ref,152.0,4.923143e+04,3.803640e+04,2.030475e+03,2.096388e+04,3.757136e+04,6.943005e+04,1.325694e+05


In [ ]:
#데이터 전처리 
#데이터 마다 특성이 달라 이를 정규화(normalization)하는 과정이 있어야 효율적 ==> 논문에서는 정규분포로 평균 0 분산 1로 정규화 (standard normal distribution)
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
X_train = norm(X_train)
X_test = norm(X_test)

In [ ]:
#모델 초기화
keras.backend.clear_session()

In [ ]:
#paper
# network is initialized with ran- domly assigned weights.==> he_uniform으로 교환
# learning rate 0.001
# L2 regularization parameter (αr) is set to 0.001
# optimization algorithm used is Adam instead of standard gradient descent algorithm
# The exponential decay rate of the first and second moments is specified to be 0.9 and 0.999
# MSE loss // maximum iteration number no meet tolerance (0.001) is 100  
# batch size 200
#model.add(BatchNormalization())  
###############################################################################

# Multilayer Perceptron (MLP) 생성

model = Sequential()

# 첫 번째 Layer (Input layer)
model.add(Dense(75, input_shape = (19, )))
model.add(Activation('relu'))

# 두 번째 Layer (Hidden layer 1)
# 두 번째 Layer부터는 output_dim만 설정하면 된다
# input_dim은 이전 레이어의 output_dim과 같다고 가정함
model.add(Dense(70))  
model.add(Activation('relu')) 

# 세 번째 Layer (Hidden layer 2)
model.add(Dense(60))  
model.add(Activation('relu'))

# 네 번째 Layer (Hidden layer 3)
model.add(Dense(50))   
model.add(Activation('relu'))

# 다섯 번째 Layer (Hidden layer 4)
model.add(Dense(30))  
model.add(Activation('relu'))

# 여섯 번째 Layer (Hidden layer 5)
model.add(Dense(20)) 
model.add(Activation('relu'))

# 일곱 번째 Layer (Hidden layer 6)
model.add(Dense(10)) 
model.add(Activation('relu'))

# 여덟 번째 Layer (Output layer)
model.add(Dense(1))  

# Cost function 및 Optimizer 설정
# Adam optimizer 사용

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0, amsgrad=False)
model.compile(optimizer = adam, loss = 'mse', metrics = ['mae','mse'])

model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 75)                1500      
_________________________________________________________________
activation (Activation)      (None, 75)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 70)                5320      
_________________________________________________________________
activation_1 (Activation)    (None, 70)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 60)                4260      
_________________________________________________________________
activation_2 (Activation)    (None, 60)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                3

In [ ]:
#simple model test
example_batch = X_train[:10]
example_result = model.predict(example_batch)
example_result

array([[-0.00284025],
       [-0.00986149],
       [-0.01117984],
       [-0.01255235],
       [-0.0111456 ],
       [-0.01324963],
       [-0.01530338],
       [-0.01738305],
       [-0.01678724],
       [-0.01875434]], dtype=float32)

In [ ]:
# model training
training_epochs = 1000
batch_size = 30
history=model.fit(X_train, y_train, batch_size=batch_size, epochs=training_epochs, verbose=1)

Epoch 1/1000
6/6 [==============================] - 0s 3ms/step - loss: 68460232.0000 - mae: 7227.6812 - mse: 68460232.0000
Epoch 2/1000
6/6 [==============================] - 0s 2ms/step - loss: 68452792.0000 - mae: 7227.2344 - mse: 68452792.0000
Epoch 3/1000
6/6 [==============================] - 0s 2ms/step - loss: 68438192.0000 - mae: 7226.3784 - mse: 68438192.0000
Epoch 4/1000
6/6 [==============================] - 0s 3ms/step - loss: 68404888.0000 - mae: 7224.4531 - mse: 68404888.0000
Epoch 5/1000
6/6 [==============================] - 0s 2ms/step - loss: 68341128.0000 - mae: 7220.6235 - mse: 68341128.0000
Epoch 6/1000
6/6 [==============================] - 0s 2ms/step - loss: 68202720.0000 - mae: 7212.5552 - mse: 68202720.0000
Epoch 7/1000
6/6 [==============================] - 0s 2ms/step - loss: 67930504.0000 - mae: 7196.1948 - mse: 67930504.0000
Epoch 8/1000
6/6 [==============================] - 0s 3ms/step - loss: 67330312.0000 - mae: 7161.0371 - mse: 67330312.0000
Epoch 9/

In [ ]:
# Model evaluation using test set
import math 
y_pr=model.predict(X_train)
y_exp= np.zeros((len(y_train),1))
for i in range(len(y_train)):
    y_exp[i][0]=y_train[i]

def MAE(x,y):
    N=len(x)
    tot=0.0
    for i in range(N):
        del_abs= abs(x[i][0]-y[i][0])
        val= del_abs/y[i][0]
        tot=tot+val
    return ((tot/N)*100)

def R2(x,y):
    N=len(x)
    up=0.0
    dn=0.0
    tot=0.0
    for j in range(N):
        tot= tot+ y[j][0]
    avg= tot/N
    for i in range(N):
        up= up+ (x[i][0]-y[i][0])**2
        dn= dn+ (y[i][0]-avg)**2
    return (1-(up/dn))
print("MAE:",MAE(y_pr,y_exp))
print("R2:",R2(y_pr,y_exp))

,loss,mae,mse,epoch
995,132925.218750,240.852615,132925.218750,995
996,134447.843750,254.985504,134447.843750,996
997,133794.093750,259.158752,133794.093750,997
998,146975.015625,268.773956,146975.015625,998
999,151672.031250,264.064026,151672.046875,999


In [ ]:
# predict h using model
# X_unknown=.... // 이는 x_train 과 같은 19개의 parameter가 있는 (1,19) 형태여야함
y_pr=model.predict(X_train)

MAE: 6.460625275349323
R2: 0.9930761866524949


In [ ]:
#save model (format: h5) number를 입력해 구분하자
model.save('my_model.h5') 
